Як я написав в файлі з аналізом чомусь маски просто прикріпляються до неправильних місць на карті

Але я постарався написати код з моделюванням

Якщо можна підсказку як виправити маски я готовий продовжити роботу, дякую

In [1]:
import os

import rasterio
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize

import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, Polygon
from shapely.ops import cascaded_union

import numpy as np
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, UpSampling2D, Concatenate, Input
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.utils import Sequence

import optuna

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
raster_path = "/kaggle/input/quantum-soil-erosion-detection/T36UXV_20200406T083559_TCI_10m.jp2"
src = rasterio.open(raster_path, "r", driver="JP2OpenJPEG")
raster_img = src.read()
raster_meta = src.meta
raster_img = reshape_as_image(raster_img)

In [3]:
train_df = gpd.read_file("/kaggle/input/quantum-soil-erosion-detection/Masks_T36UXV_20190427.shp")

# let's remove rows without geometry
train_df = train_df[train_df.geometry.notnull()]

# assigning crs
train_df.crs = {'init' :'epsg:4267'}

#transforming polygons to the raster crs
train_df = train_df.to_crs({'init' : raster_meta['crs']['init']})

/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [4]:
def poly_from_utm(polygon, transform):
    poly_pts = []
    
    # make a polygon from multipolygon
    poly = cascaded_union(polygon)
    for i in np.array(poly.exterior.coords):
        
        poly_pts.append(~transform * tuple(i))
        
    # make a shapely Polygon object
    new_poly = Polygon(poly_pts)
    return new_poly


poly_shp = []
im_size = (src.meta['height'], src.meta['width'])
for num, row in train_df.iterrows():
    if row['geometry'].geom_type == 'Polygon':
        poly = poly_from_utm(row['geometry'], src.meta['transform'])
        poly_shp.append(poly)
    else:
        for p in row['geometry']:
            poly = poly_from_utm(p, src.meta['transform'])
            poly_shp.append(poly)

mask = rasterize(shapes=poly_shp,
                 out_shape=im_size)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  import sys


In [5]:
# Define batch size
batch_size = 32

val_split = int(10980*0.8)
print(val_split)

# Load image and mask using load_img from Keras
train_image = raster_img[:val_split]
train_mask = mask[:val_split, :, None]

val_image = raster_img[val_split:]
val_mask = mask[val_split:, :, None]

8784


In [6]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, activation='relu', padding='same')(inputs)
    x = Conv2D(num_filters, 3, activation='relu', padding='same')(x)
    return x

def unet(input_shape=(256, 256, 3)):
    inputs = Input(shape=input_shape)

    conv1 = conv_block(inputs, 64)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = conv_block(pool1, 128)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = conv_block(pool2, 256)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = conv_block(pool3, 512)
    drop4 = Dropout(0.5)(conv4)

    up5 = Conv2D(256, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(drop4))
    merge5 = Concatenate(axis=3)([conv3, up5])

    conv6 = conv_block(merge5, 256)
    up7 = Conv2D(128, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = Concatenate(axis=3)([conv2, up7])

    conv8 = conv_block(merge7, 128)
    up9 = Conv2D(64, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv8))
    merge9 = Concatenate(axis=3)([conv1, up9])

    conv10 = conv_block(merge9, 64)

    outputs = Conv2D(1, 1, activation='sigmoid')(conv10)

    model = Model(inputs=inputs, outputs=outputs)

    return model

In [7]:
def dice_coefficient(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1, 2, 3])
    union = K.sum(y_true, axis=[1, 2, 3]) + K.sum(y_pred, axis=[1, 2, 3])
    return K.mean((2. * intersection + smooth) / (union + smooth), axis=0)

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

In [8]:
def random_crop(image, mask, crop_size):
    height, width = image.shape[:2]
    dy, dx = crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    image_crop = image[y:(y+dy), x:(x+dx), :]
    mask_crop = mask[y:(y+dy), x:(x+dx), :]
    return image_crop, mask_crop

def generate_random_batch(datagen, image_arr, mask_arr, batch_size, crop_size):
    while True:
        batch_images = np.zeros((batch_size, crop_size[0], crop_size[1], 3))
        batch_masks = np.zeros((batch_size, crop_size[0], crop_size[1], 1))
        for i in range(batch_size):
            image_crop, mask_crop = random_crop(image_arr, mask_arr, crop_size)
            image_crop, mask_crop = next(datagen.flow(image_crop[np.newaxis], mask_crop[np.newaxis]))
            batch_images[i] = image_crop
            batch_masks[i] = mask_crop
        yield batch_images, batch_masks

In [9]:
train_datagen = ImageDataGenerator(
        rescale=1./255)

train_generator = generate_random_batch(train_datagen, train_image, train_mask, batch_size, (256, 256))

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = generate_random_batch(val_datagen, val_image, val_mask, batch_size, (256, 256))

steps_per_epoch = int(np.ceil(1. * 10980 / 256) * np.ceil(1. * 10980 / 256) / batch_size)

In [12]:
model = unet()
model.compile(optimizer=Adam(lr=0.5),
              loss=dice_loss,
              metrics=[dice_coefficient])

history = model.fit(train_generator, 
          epochs=10, 
          steps_per_epoch=steps_per_epoch,
          validation_data=val_generator,
          validation_steps=steps_per_epoch)

In [ ]:
def objective(trial):
    initial_lr = trial.suggest_loguniform('lr', 1e-3, 1e-1)
    decay_rate = trial.suggest_uniform('decay_rate', 0.1, 0.9)
    decay_step = 1
    
    def lr_schedule(epoch):
        return initial_lr * decay_rate ** (epoch // decay_step)

    lr_scheduler = LearningRateScheduler(lr_schedule)

    model = unet()
    model.compile(optimizer=Adam(lr=initial_lr),
                  loss=dice_loss,
                  metrics=[dice_coefficient])

    history = model.fit(train_generator, 
              epochs=1, 
              steps_per_epoch=steps_per_epoch,
              validation_data=val_generator,
              validation_steps=steps_per_epoch,
              callbacks=[lr_scheduler])

    if history.history['val_loss'][0] > 0.5:
        raise optuna.TrialPruned()
    
    history = model.fit(train_generator, 
              epochs=3, 
              steps_per_epoch=steps_per_epoch,
              validation_data=val_generator,
              validation_steps=steps_per_epoch,
              callbacks=[lr_scheduler])
        
    return history.history['val_loss'][-1]

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

print("Best trial: ", study.best_trial.params)

In [ ]:
train_generator = generate_random_batch(train_datagen, train_image, train_mask, batch_size, (256, 256))
while 1:
    images, masks = next(train_generator)
    if np.sum(masks) > 10000:
        break

predicted = model.predict(images)
print(np.sum(predicted))
for i in range(32):
    print('image:')
    plt.imshow(images[i])
    plt.show()
    print('predicted mask:')
    plt.imshow(predicted[i])
    plt.show()
    print('actual mask:')
    plt.imshow(masks[i])
    plt.show()